In [4]:
# Notebook til at se på SSI's data for kontakttal
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "Dark2"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
# import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [1]:
print('Make sure to run "get_data" first, so the most recent data is used')

Make sure to run "get_data" first, so the most recent data is used


In [13]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

d:\Pandemix\Github\DanskeData/ssi_data\SSI_data_2021-04-13
2021-04-13 00:00:00


In [144]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldf = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
df = fulldf.iloc[:-2,:].copy()

In [145]:

dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
df["PosPct"] = pd.to_numeric(df["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
df["Date"] = df["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ","PosPct"]
for name in rows_to_fix_period_in:
    df[name] = pd.to_numeric(df[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [150]:
# Plot different things
firstDate = np.datetime64('2020-03-01') # All tests
firstDate = np.datetime64('2021-02-01') # Antigen test
today = date.today()

# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(df.Date,df.NewPositive,label='Nye positive')
# ax1.plot(df.Date,df.Tested)

ax2.plot(df.Date,100*np.divide(df.NewPositive,df.Tested),label='Positiv procent')
ax2.plot(df.Date,0.1*df.PosPct,label='Positiv procent (Afrundet)')

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)

ax1.set_xlim([firstDate,today])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Figure to compare antigen positive procent with PCR

In [170]:
# Plot different things
firstDate = np.datetime64('2020-03-01') # All tests
firstDate = np.datetime64('2021-02-01') # Antigen test
today = date.today()

# fig,ax1 = plt.subplots(1,1)
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

# ax1.plot(df.Date,df.NewPositive,label='Nye positive')
ax1.plot(df.Date,df.Tested,label='Tested, antigen')
ax1.plot(dfPCR.Date,dfPCR.Tested,label='Tested, PCR')

ax2.plot(df.Date,df.NewPositive,label='New positive, antigen')
ax2.plot(dfPCR.Date,dfPCR.NewPositive,label='New positive, PCR')

posPctAG = 100*np.divide(df.NewPositive,df.Tested)
posPctPCR = 100*np.divide(dfPCR.NewPositive,dfPCR.Tested)
ax3.plot(df.Date,posPctAG,label='Positiv procent, antigen')
ax3.plot(dfPCR.Date,posPctPCR,label='Positiv procent, PCR')
# ax2.plot(df.Date,df.PosPct,label='Positiv procent (Afrundet)')
# ax2.plot(dfPCR.Date,0.1*dfPCR.PosPct,label='Positiv procent (Afrundet)')

backToShow = -30

ax1.set_ylim(bottom=0)
ax2Top = np.max(dfPCR.NewPositive[backToShow:])*1.1
ax2.set_ylim(bottom=0,top=ax2Top)
ax3Top =np.max(posPctPCR[backToShow:])*1.1
ax3.set_ylim(bottom=0,top=ax3Top)

ax1.legend()
ax2.legend()
ax3.legend()

ax1.set_xlim([firstDate,today])
ax1.set_xlim([firstDate,today-np.timedelta64(2,'D')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))

plt.tight_layout()

if saveFigures:
    fig.savefig('figs/PosPctComparison')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [171]:
# df.NewPositive